# **NLP - Sentiment Analysis Division Sigma Competition (Our Efforts)**

## *1.0 Importing necessary libraries*

In [ ]:
# Standard imports - sklearn and transformer imports will be handled later
import tensorflow as tf
from tensorflow import keras
from google.colab import drive
import os
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import numpy as np
import pandas as pd

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


### 1.1 Import and save Training Data from Google Drive

In [ ]:
from google.colab import files
data_to_load = files.upload()

Saving training_data.csv to training_data.csv


In [ ]:
import io
df = pd.read_csv(io.StringIO(data_to_load["training_data.csv"].decode("utf-8")))  

In [ ]:
#We pickle the dataframe to make it easier to load for late uses.
df.to_pickle('train-df.csv') 

In [ ]:
df = pd.read_pickle('train-df.csv')

We define and use a quick lambda function for visualizations 

### 1.2 Setting Up Sentiments Label and Data Frame

In [ ]:
#Uses the sentiment reading in order to add a sentiment label to the data frame
def encoded_to_label(sentiment_encode):
  if sentiment_encode == 1:
    return "Positive"
  else:
    return "Negative"

df["Sentiment Label"] = df["Sentiment"].apply(lambda x: encoded_to_label(x))

### 1.3 Displaying Data before Preprocessing

In [ ]:
df

,ID,User,Text,Sentiment,Sentiment Label
0,864192,Carly_FTS,I *heart* filling up @dennisschaub desk 1 it...,1,Positive
1,523691,Open_Sourcing,"#SocioMat - people create prettier, younger an...",1,Positive
2,584154,xxcharlx,no way i dont want the tour to end,0,Negative
3,1527961,andreapuddu,@HemalRadia Hi Amazing Brother! Sending Limitl...,1,Positive
4,28609,umbec,@flockmaster they are chocolate,1,Positive
...,...,...,...,...,...
999995,1366175,b13thy,@midderhonz i'm good.. off to buy an electric ...,1,Positive
999996,681828,HeyyitsALison,@StaceyPaha i know..for youngerr boys..what am...,1,Positive
999997,488988,sleepycove,I can't belive it I just got asked for an auto...,1,Positive
999998,985613,AmyyXD,i am putting my bb in the fridge so it cant di...,0,Negative


In [ ]:
# Checking for empty values
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 5 columns):
 #   Column           Non-Null Count    Dtype 
---  ------           --------------    ----- 
 0   ID               1000000 non-null  int64 
 1   User             1000000 non-null  object
 2   Text             1000000 non-null  object
 3   Sentiment        1000000 non-null  int64 
 4   Sentiment Label  1000000 non-null  object
dtypes: int64(2), object(3)
memory usage: 38.1+ MB


## *2.0 Preprocessing Data*

### 2.1 Formating The Text for the Bert or DistilBert Model

In [ ]:
# Useful libraries
import re 
import unicodedata 
import string

# Removing punctuation with regular expressions
def clean_sentence(text):
    text = unicodedata.normalize('NFKD', text)
    text = text.strip(', , ,')
    text = text.strip('\t\n') 
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', '', text)
    return text

In [ ]:
# Applying the data cleaning to the pandas dataframe
df_revised =  pd.DataFrame(df['Text'].apply(lambda x: clean_sentence(x)))
df_revised['Sentiment'] = df['Sentiment']

In [ ]:
# Before we begin encoding the text, we need to split it up into a training and validation set
# We use scikit-learn's train_test_split for this
from sklearn.model_selection import train_test_split
train_df, valid_df = train_test_split(df_revised, test_size = 0.2)

In [ ]:
# Necessary pip command to use HuggingFace's transformer library and toolkit
!pip install transformers==3.0.0

     |████████████████████████████████| 757kB 4.8MB/s 
     |████████████████████████████████| 1.1MB 23.2MB/s 
     |████████████████████████████████| 3.0MB 40.8MB/s 
     |████████████████████████████████| 890kB 43.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=361ebd0cd58b051e429c233fbbfd6ab0f6448afbfe35aff9aa636368736331c7
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


Before we feed the data to the BERT or DistilBert model, we must preprocess it. First, we must tokenize the data, add start-of-sentence and end-of-sentence tokens, pad the data, and mask it. This will be done with the transformer tokenizer's batch_encode_plus(method), which enables us to do all of this at once.
Then, we must

### 2.2 Tokenizing, Padding, and Masking

In [71]:
# Necessary imports
import transformers 
from transformers import BertTokenizer, TFBertModel

BERT_MODEL = 'bert-base-cased'
 
test = 160

 # Using the names of the available models from the pretrained_models documentation
tokenizer = BertTokenizer.from_pretrained(BERT_MODEL)

# To silence Colab warnings
import logging
logging.basicConfig(level=logging.ERROR)

# Encoding the data
def encode_data(text, tokenizer, testlen=160): 
  encoded =  tokenizer.batch_encode_plus(
      text,
      add_special_tokens=True,
      return_attention_masks=True, 
      return_token_type_ids=False,   
      pad_to_max_length=True,
      max_length=test
  ) 
  return np.array(encoded['input_ids'])

# Applying the encoding to the training and validation data
train_encoded_data = encode_data(train_df.Text.values, tokenizer, testlen = 160)
valid_encoded_data = encode_data(valid_df.Text.values, tokenizer, testlen = 160)  

### 2.3 Converting to Tensorflow Dataset

We convert the pandas series that were output previously and convert it to TensorFlow datasets with the from_tensor_slices(method).

In [72]:
y_train = train_df.Sentiment.values
y_valid = valid_df.Sentiment.values
 
BATCH_SIZE = 32

ds_train = (tf.data.Dataset 
.from_tensor_slices((train_encoded_data, y_train)) 
.repeat()
.shuffle(512)
.batch(BATCH_SIZE)
.prefetch(tf.data.experimental.AUTOTUNE))


ds_valid = (tf.data.Dataset 
.from_tensor_slices((valid_encoded_data, y_valid)) 
.repeat()
.shuffle(512)
.batch(BATCH_SIZE)
.prefetch(tf.data.experimental.AUTOTUNE))

## *3.0 Training the Model*

The Bert model training came out to about 300+ hours for us for some reason and we tried to spend a very long time to fix it but we werent able to. Alot of PAIN :( We decided to change the model to distilbert however the training was still way to long at about 4+hours per epoch. This is weird for even the distilbert model as it should be able to be trained fully in under an hour for this training data set. We tried our best to cut down the training time but kept getting training errors at the end of when an epoch was about to finish. Trying to avoid the training errors we also added checkpoints but those didn't end up working lol!!!! In the end we tried our best and even though we werent able to succeed we managed to learn alot about NLP models and their implementation. 

In [ ]:
 #setting up checkpoint call backs for the modes training 
 checkpoint_cb = keras.callbacks.ModelCheckpoint('sigma_bert_model.h5', save_best_only=True)  

Implementing the distilbert model after the bert model didnt work

In [ ]:
from transformers import TFDistilBertForSequenceClassification

model=TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-cased') 
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

In [ ]:
Distilbert model summary

In [ ]:
model.summary()

Model: "tf_distil_bert_for_sequence_classification_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
distilbert (TFDistilBertMain multiple                  65190912  
_________________________________________________________________
pre_classifier (Dense)       multiple                  590592    
_________________________________________________________________
classifier (Dense)           multiple                  1538      
_________________________________________________________________
dropout_139 (Dropout)        multiple                  0         
Total params: 65,783,042
Trainable params: 65,783,042
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Uncomment the next line to restore the checkpoint
#This sadly didnt end up working :(
# model = keras.models.load_model('sigma_bert_model.ht')

Tried to train the data below, but due to the long training times either human errors would happen or the training would just not work after one epoch. Spent a long time tring to change the hyperparamters but only the base model was properly training for atleast one epoch before breaking. PAIN :(

In [ ]:
number_steps = 800000/BATCH_SIZE
history = model.fit(ds_train, steps_per_epoch = number_steps, validation_data = ds_valid, epochs = 3, callbacks = [checkpoint_cb]) 